IMPORT

In [ ]:
import os
import pandas as pd
from Libraries import Trainer
from transformers import pipeline
from Libraries import Processor as pc
from Libraries.Sorter import ArticleSorter
# from Libraries.Crawler import CategoryValidator, UrlCollector, ArticleCrawler

CRAWL CONFIG

In [ ]:
type_dict = pc.load_json("Resource/categories.json")

my_config = {
    "BASE_URL": "https://vnexpress.net",

    "MIN_YEAR": 2020,
    "MIN_WORDS": 200,
    "MAX_WORDS": 1000,
    
    "TARGET_ARTICLES_PER_SUBTYPE": 5,
    "MAX_CONCURRENT_WORKERS": 6,
    "VALIDATION_ARTICLES_COUNT": 5,
    "PROGRESS_TIMEOUT": 10,
    "ARTICLE_TIMEOUT": 10,
    "MAX_CONSECUTIVE_FAILURES": 3,
    "URL_MAX_SUBCATEGORY_FAILURES": 3, 
    "ARTICLE_TIMEOUT": 5,
    
    "TYPE_DICT": type_dict
}

resource_dir = "Resource"
database_dir = "Database"
pageName = "VNExpress"

CATE_FILE = f"{resource_dir}/{pageName}_CATE.json"
DICT_FILE = f"{resource_dir}/{pageName}_DICT.json"

URLS_PATH = f"{resource_dir}/{pageName}_URLS"
JSON_PATH = f"{database_dir}/JSON/{pageName}"
XLSX_PATH = f"{database_dir}/XLSX/{pageName}"

os.makedirs(resource_dir, exist_ok=True)
os.makedirs(os.path.join(database_dir, "JSON"), exist_ok=True)
os.makedirs(os.path.join(database_dir, "XLSX"), exist_ok=True)

validated_dict = pc.load_json(DICT_FILE)
categories_to_crawl = [
    'thoi-su', 
    'the-gioi', 
    'kinh-doanh', 
    'bat-dong-san', 
    'khoa-hoc', 
    'giai-tri', 
    'the-thao', 
    'phap-luat', 
    'giao-duc', 
    'suc-khoe', 
    'doi-song', 
    'oto-xe-may'
]

CRAWL FUNCTIONS

In [ ]:
# --- Giai đoạn 1: Lấy danh sách chuyên mục hợp lệ ---
def getCategories():
    validator = CategoryValidator(config=my_config)
    valid_categories = validator.run()
    pc.replace_json(valid_categories, DICT_FILE)

In [ ]:
# --- Giai đoạn 2: Thu thập URL ---
def getDict():
    for category_name in categories_to_crawl:

        URLS_FILE = f"{URLS_PATH}_{category_name}.json"
        JSON_FILE = f"{JSON_PATH}_{category_name}.jsonl"

        print("\n" + "="*50)
        print(f"BẮT ĐẦU LẤY URL CHO CATEGORY: {category_name.upper()}")
        print("="*50)

        # 1. Lấy danh sách TẤT CẢ các URL đã tồn tại từ cả 2 nguồn
        urls_in_url_file = pc.get_urls_from_url_file(URLS_FILE)
        urls_in_jsonl_file = pc.get_existing_article_urls(JSON_FILE)
        all_existing_urls = urls_in_url_file.union(urls_in_jsonl_file)
        
        print(f"Đã tìm thấy {len(all_existing_urls)} URL đã tồn tại cho category này.")

        # 2. Chạy collector để thu thập tất cả URL có thể có
        url_collector = UrlCollector(config=my_config)
        all_possible_urls = url_collector.run(
            valid_subcategories=validated_dict,
            categories_to_process=[category_name]
        )

        # 3. Lọc ra chỉ những URL thực sự mới
        new_urls_info = [
            info for info in all_possible_urls 
            if info['url'] not in all_existing_urls
        ]
        
        print(f"Thu thập được {len(new_urls_info)} URL mới.")

        # 4. Ghi lại file URLS với dữ liệu cũ + mới
        if new_urls_info:
            existing_urls_info = pc.load_json(URLS_FILE)
            combined_urls_info = existing_urls_info + new_urls_info
            pc.replace_json(combined_urls_info, URLS_FILE)
    
# === END ===

In [ ]:
# --- Giai đoạn 3: Crawl nội dung bài viết ---
def runCrawl():
    for category_name in categories_to_crawl:

        URLS_FILE = f"{URLS_PATH}_{category_name}.json"
        JSON_FILE = f"{JSON_PATH}_{category_name}.jsonl"

        print("\n" + "="*50)
        print(f"BẮT ĐẦU CRAWL CHO CATEGORY: {category_name.upper()}")
        print("="*50)

        urls_for_category = pc.load_json(URLS_FILE)
        if not urls_for_category:
            print("Không có URL nào trong file để crawl. Bỏ qua.")
            continue

        # 1. Xác định các URL đã được xử lý từ trước
        existing_urls_in_jsonl = pc.get_existing_article_urls(JSON_FILE)
        print(f"Tìm thấy {len(existing_urls_in_jsonl)} bài viết đã tồn tại trong file {JSON_FILE}.")
        
        # 2. Chạy crawler
        article_crawler = ArticleCrawler(config=my_config)
        new_articles, successfully_crawled_urls = article_crawler.run(
            urls_to_crawl=urls_for_category,
            category=category_name,
            existing_article_urls=existing_urls_in_jsonl
        )
        
        # 3. Lưu dữ liệu mới nếu có
        if new_articles:
            pc.save_jsonl(new_articles, JSON_FILE)
            print(f"Đã lưu {len(new_articles)} bài báo mới vào {JSON_FILE}.")

        # 4. Xóa các URL đã được xử lý khỏi file URLS_FILE
        processed_urls = existing_urls_in_jsonl.union(set(successfully_crawled_urls))
        if processed_urls:
            # Lọc ra danh sách các URL còn lại
            remaining_urls = [
                info for info in urls_for_category 
                if info['url'] not in processed_urls
            ]
            
            # Ghi đè lại file URLS_FILE
            pc.replace_json(remaining_urls, URLS_FILE)
            print(f"Đã xóa {len(processed_urls)} URL đã xử lý. Còn lại {len(remaining_urls)} URL trong hàng đợi.")

# === END ===

RESIGN DATA FUNCTIONS

In [ ]:
# --- Giai đoạn 4: Sort bài viết ---
def sortCrawl():
    for category_name in categories_to_crawl:

        URLS_FILE = f"{URLS_PATH}_{category_name}.json"
        JSON_FILE = f"{JSON_PATH}_{category_name}.jsonl"

        print("\n" + "="*50)
        print(f"BẮT ĐẦU SORT CHO CATEGORY: {category_name.upper()}")
        print("="*50)

        # --- Sắp xếp URLS_FILE ---
        urls_data = pc.load_json(URLS_FILE)
        if urls_data:
            sorted_urls = pc.heapSort(urls_data, key_func=pc.get_url_key)
            pc.replace_json(sorted_urls, URLS_FILE)

        # --- Sắp xếp JSON_FILE ---
        all_articles = pc.load_jsonl(JSON_FILE)
        if all_articles:
            sorter = ArticleSorter(categories_file_path=CATE_FILE)
            sorted_articles = sorter.sort_and_deduplicate(all_articles)
            pc.replace_jsonl(sorted_articles, JSON_FILE)
        
        print(f"-> Đã hoàn tất sắp xếp cho category: {category_name}")

# === END ===

In [ ]:
# --- Giai đoạn 5: Tổng hợp và Hoàn thiện Dữ liệu (Logic cuối cùng, tự động) ---
def finalizeData():
    # Định nghĩa đường dẫn cho các file master
    MASTER_JSONL_FILE = f"{JSON_PATH}.jsonl"
    MASTER_JSON_FILE = f"{JSON_PATH}.json" 
    MASTER_XLSX_FILE = f"{XLSX_PATH}.xlsx"

    categories_dict = pc.load_json(CATE_FILE)
    if not categories_dict:
        return
    
    all_categories = list(categories_dict.keys())

    # Xóa các file tổng hợp cũ để bắt đầu lại từ đầu
    for f in [MASTER_JSONL_FILE, MASTER_JSON_FILE]:
        if os.path.exists(f):
            os.remove(f)

    # 1. Gộp dữ liệu từ các file JSONL của tất cả category tìm được
    print("\n--- Bước 1: Gộp dữ liệu từ các file category ---")
    for category_name in all_categories: # Sử dụng danh sách vừa đọc được
        JSON_FILE = f"{JSON_PATH}_{category_name}.jsonl"
        # Chỉ xử lý nếu file của category đó tồn tại
        if os.path.exists(JSON_FILE):
            articles_data = pc.load_jsonl(JSON_FILE)
            if articles_data:
                pc.save_jsonl(articles_data, MASTER_JSONL_FILE)

    # 2. Sắp xếp và xóa trùng lặp file tổng hợp
    print("\n--- Bước 2: Sắp xếp và dọn dẹp file tổng hợp ---")
    all_merged_articles = pc.load_jsonl(MASTER_JSONL_FILE)
    if all_merged_articles:
        sorter = ArticleSorter(categories_file_path=CATE_FILE)
        final_sorted_articles = sorter.sort_and_deduplicate(all_merged_articles)

        # 3. Ghi kết quả ra cả 2 định dạng file
        print("\n--- Bước 3: Ghi file tổng hợp ở cả 2 định dạng (.jsonl và .json) ---")
        pc.replace_jsonl(final_sorted_articles, MASTER_JSONL_FILE)
        pc.replace_json(final_sorted_articles, MASTER_JSON_FILE)

        # 4. Xuất file Excel cuối cùng
        print("\n--- Bước 4: Xuất file Excel tổng hợp ---")
        pc.convert_to_xlsx(MASTER_JSON_FILE, MASTER_XLSX_FILE)

# === END ===

TRAINING CONFIG

In [ ]:
# === CONFIG ===
JSON_PATH = f"{database_dir}/Data/TrainData"

training_config = {
    # --- Đường dẫn và tên model ---
    "DATA_JSONL_FILE": f"{JSON_PATH}.jsonl",
    "MODEL_CHECKPOINT": "vinai/bartpho-syllable",
    "OUTPUT_MODEL_DIR": "Models/bartpho-summarizer",
    
    # --- Hyperparameters ---
    "MAX_INPUT_LENGTH": 1024,
    "MAX_TARGET_LENGTH": 256,
    "BATCH_SIZE": 4,
    "NUM_TRAIN_EPOCHS": 3,
    "LEARNING_RATE": 3e-5,
    "WEIGHT_DECAY": 0.01,
}

TRAINING FUNCTION

In [ ]:
# === TRAIN ===
def modelTrain():
    summarizer_trainer = Trainer.SummarizationTrainer(config=training_config)
    summarizer_trainer.run()

In [ ]:
# === TEST ===
def modelTest():
    fine_tuned_model_path = training_config["OUTPUT_MODEL_DIR"] 
    summarizer_pipeline = pipeline("summarization", model=fine_tuned_model_path)

    # Lấy một bài báo từ dữ liệu để tóm tắt thử
    df = pd.read_json(training_config["DATA_JSONL_FILE"], lines=True)
    sample_text = df.iloc[50]["article"]
    # sample_text = df.iloc[50]["content"]

    print("--- VĂN BẢN GỐC ---")
    print(sample_text)
    print("\n" + "="*50 + "\n")

    print("--- BẢN TÓM TẮT TỪ MODEL ---")
    summary = summarizer_pipeline(
        sample_text,
        max_length=training_config["MAX_TARGET_LENGTH"],
        min_length=50,
        do_sample=False,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    print(summary[0]['summary_text'])

RUN

In [ ]:
# getCategories()
# getDict()
# runCrawl()
# sortCrawl()
# finalizeData()

In [ ]:
modelTrain()
modelTest()

In [ ]:
# Main.ipynb

# !python Libraries/API.py